<a href="https://colab.research.google.com/github/Tekleab15/Regularized_Auto_Encoder/blob/main/RAE_implementation_using_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  **Regularized Autoencoder - RAE Using MNIST dataset **

*Implementation of RAE(Regularized Autoencoders) as per the specification on the paper THE NEURAL CODING FRAMEWORK FOR LEARNING GENERATIVE MODELS  *

Importing required libraries and methods

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, InputLayer, BatchNormalization, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import Callback
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.initializers import RandomNormal
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

Preprocess dataset according to the specification provided in the paper

In [2]:
def preprocess_dataset(x_train, x_test):
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    x_train = (x_train > 0.5).astype('float32')
    x_test = (x_test > 0.5).astype('float32')
    x_train = x_train.reshape(-1, 784)
    x_test = x_test.reshape(-1, 784)
    return x_train, x_test

Loading and Preprocess the dataset from the keras datasets

In [3]:
# Load and preprocess the MNIST dataset, including labels
(x_train_raw, y_train), (x_test_raw, y_test) = mnist.load_data()
x_train, x_test = preprocess_dataset(x_train_raw, x_test_raw)

Designing the model Architecture as per the specification in the paper

In [11]:
input_shape = (784,)
latent_dim = 20
hidden_layer_size = 360

# Encoder section
encoder = Sequential(name="encoder")
encoder.add(InputLayer(shape=input_shape))
encoder.add(Dense(hidden_layer_size, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))
encoder.add(BatchNormalization())
encoder.add(Dropout(0.2))
# encoder.add(Dense(hidden_layer_size, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))
# encoder.add(Dense(hidden_layer_size, activation='relu'))
encoder.add(Dense(hidden_layer_size, activation='relu'))
encoder.add(BatchNormalization())
encoder.add(Dropout(0.2))
encoder.add(Dense(latent_dim, activation='sigmoid'))

# Decoder section
decoder = Sequential(name="decoder")
decoder.add(InputLayer(shape=(latent_dim,)))
# decoder.add(Dense(hidden_layer_size, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05), kernel_regularizer=regularizers.l2(1e-6)))
# decoder.add(Dense(hidden_layer_size, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05), kernel_regularizer=regularizers.l2(1e-6)))
decoder.add(Dense(hidden_layer_size, activation='relu', kernel_regularizer=regularizers.l2(1e-2)))
decoder.add(BatchNormalization())
decoder.add(Dropout(0.2))
decoder.add(Dense(hidden_layer_size, activation='relu', kernel_regularizer=regularizers.l2(1e-2)))
decoder.add(BatchNormalization())
decoder.add(Dropout(0.2))
decoder.add(Dense(input_shape[0], activation='sigmoid'))
decoder.add(Reshape(input_shape))

rae = Sequential([encoder, decoder], name="RAE")

In [12]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)

In [15]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.005, clipnorm=5.0)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True, clipnorm=5)
rae.compile(optimizer=optimizer, loss='binary_crossentropy')
print(rae.summary())

Model: "RAE"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ encoder (Sequential)                 │ (None, 20)                  │         422,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Sequential)                 │ (None, 784)                 │         423,424 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 846,084 (3.23 MB)

 Trainable params: 843,204 (3.22 MB)

 Non-trainable params: 2,880 (11.25 KB)

None


**Training the RAE model**

In [16]:
rae.fit(x_train, x_train, epochs=50, batch_size=200, validation_data=(x_test, x_test))

Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - loss: 3.8930 - val_loss: 1.9043
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - loss: 1.6408 - val_loss: 1.0124
Epoch 3/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - loss: 0.9392 - val_loss: 0.7322
Epoch 4/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.7014 - val_loss: 0.6149
Epoch 5/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - loss: 0.6000 - val_loss: 0.5310
Epoch 6/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - loss: 0.5235 - val_loss: 0.4500
Epoch 7/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - loss: 0.4411 - val_loss: 0.3682
Epoch 8/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 0.3622 - val_loss: 0.3025
Epoch 9/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - loss: 0.3022 - val_loss: 0.2586
Epoch 10/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - loss: 0.2631 - val_loss: 0.2307
Epoch 11/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - loss: 0.2389 - val_loss: 0.2130
Epoch 12/50
300/300 ━━━━━━━━━━

In [17]:
# Implementing and fitting the GMM model
z_train = encoder.predict(x_train)
gmm = GaussianMixture(n_components=75, covariance_type='full').fit(z_train)

sampled_latent = gmm.sample(n_samples=5000)[0]
generated_samples = decoder.predict(sampled_latent)

log_likelihood = gmm.score_samples(z_train)
average_log_likelihood = np.mean(log_likelihood)
print("Average Log Likelihood: ", average_log_likelihood)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Average Log Likelihood:  7.3144807655190505


In [18]:
# Encode test data
z_test = encoder.predict(x_test)

# Calculate the log likelihood of the latent representations
log_likelihood_test = gmm.score_samples(z_test)
average_log_likelihood_test = np.mean(log_likelihood_test)
print("Average Log Likelihood for Test Data: ", average_log_likelihood_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Average Log Likelihood for Test Data:  6.867544435173674


In [20]:
def evaluate_model_per_sample(model, x_test, y_test, gmm):
    reconstructions = model.predict(x_test)
    bce_mean = tf.keras.losses.BinaryCrossentropy()(x_test, reconstructions).numpy()
    mse_mean = tf.keras.losses.MeanSquaredError()(x_test, reconstructions).numpy()
    bce_per_sample = bce_mean * x_test.shape[1]
    mse_per_sample = mse_mean * x_test.shape[1]

    z_test = encoder.predict(x_test)
    z_train = encoder.predict(x_train)
    z_train = z_train / np.linalg.norm(z_train, axis=1, keepdims=True)
    z_test = z_test / np.linalg.norm(z_test, axis=1, keepdims=True)

    log_reg = LogisticRegression(max_iter=1000, solver='lbfgs')
    log_reg.fit(z_train, y_train)
    y_pred = log_reg.predict(z_test)
    classification_error = 100 * (1 - np.mean(y_pred == y_test))

    return {
        "MSE per Sample": round(mse_per_sample, 2),
        "BCE per Sample": round(bce_per_sample, 2),
        "Log-Likelihood per Sample": round(average_log_likelihood_test, 2),
        "Classification Error (%)": round(classification_error, 2),
        # "Generated Samples": generated_samples
    }

evaluation_results = evaluate_model_per_sample(rae, x_test, y_test, gmm)
print(evaluation_results)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
{'MSE per Sample': 30.44, 'BCE per Sample': 102.37, 'Log-Likelihood per Sample': 6.87, 'Classification Error (%)': 14.16}
